<h1>Algorithms tests</h1>

In [1]:
import sys
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
sys.path.append('/Users/Evgeny/wonderful_programming/fuzzy-fusion-venv/fuzzy-fusion/src/')
from generator.generator import generator
from algorithm.gibbs import gibbs_sampl
from algorithm.gibbs_fuzzy import gibbs_fuzzy
from algorithm.em import em

print 'Python version ' + sys.version
print 'Pandas version ' + pd.__version__

Python version 2.7.10 (default, Oct 23 2015, 18:05:06) 
[GCC 4.2.1 Compatible Apple LLVM 7.0.0 (clang-700.0.59.5)]
Pandas version 0.17.1


In [ ]:
s_number = 12
obj_number = 50
cl_size = 2
cov_list = [0.7]*s_number
p_list = [0.7]*s_number
accuracy_list = [0.8]*s_number

accuracy_for_df = [[i, accuracy_list[i]] for i in range(s_number)]
accuracy_data = pd.DataFrame(accuracy_for_df, columns=['S', 'A'])

result_list = []
iter_list = []
for g_true in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    
    print g_true
    print '*****'
        
    for i in range(10):
        print i
        ground_truth = [random.randint(0, 1) for i in range(obj_number)]
        data, g_data = generator(cov_list, p_list, ground_truth, cl_size, g_true)
        em_d, em_it = em(data=data, accuracy=accuracy_data, truth_obj_list=ground_truth)
        while True:
            try:
                g_d, g_it = gibbs_sampl(data=data, accuracy_data=accuracy_data, truth_obj_list=ground_truth)
                gf_d, gf_it = gibbs_fuzzy(data=data, accuracy_data=accuracy_data, g_data=g_data, truth_obj_list=ground_truth)
                break
            except ZeroDivisionError:
                print 'zero {}'.fprmat(i)
        result_list.append([g_true, em_d, g_d, gf_d])
        iter_list.append([g_true,em_it, g_it, gf_it])

0.0
*****
0
1
2
3
4
5


In [9]:
# columns_d = ['g_true', 'em_d', 'g_d', 'gf_d']
# columns_it = ['g_true', 'em_it', 'g_it', 'gf_it']
# output_data = pd.DataFrame(data=result_list, columns=columns_d)
# output_iter = pd.DataFrame(data=iter_list, columns=columns_it)
# output_data.to_csv('output_data10_50_2.csv')
# output_iter.to_csv('output_iter10_50_2.csv')
# print 'Done!'

In [10]:
x = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
g_list = [1., 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0]
em, em_std = [], []
g, g_std = [], []
gf, gf_std = [], []


em_it, em_it_std = [], []
g_it, g_it_std = [], []
gf_it, gf_it_std = [], []

columns_d = ['g_true', 'em_d', 'g_d', 'gf_d']
columns_it = ['g_true', 'em_it', 'g_it', 'gf_it']
# output_data = pd.DataFrame(data=result_list, columns=columns_d)
# output_iter = pd.DataFrame(data=iter_list, columns=columns_it)
output_data = pd.read_csv('output_data10_50.csv')
output_iter = pd.read_csv('output_iter10_50.csv')
for g_true in g_list:
    data = output_data[output_data.g_true == g_true]
    iter_df = output_iter[output_iter.g_true == g_true]
    em.append(np.mean(data.em_d))
    em_std.append(np.std(data.em_d))
    g.append(np.mean(data.g_d))
    g_std.append(np.std(data.g_d))
    gf.append(np.mean(data.gf_d))
    gf_std.append(np.std(data.gf_d))
    
    em_it.append(np.mean(iter_df.em_it))
    em_it_std.append(np.std(iter_df.em_it))
    g_it.append(np.mean(iter_df.g_it))
    g_it_std.append(np.std(iter_df.g_it))
    gf_it.append(np.mean(iter_df.gf_it))
    gf_it_std.append(np.std(iter_df.gf_it))

In [11]:
f, axarr = plt.subplots()
f.set_size_inches(14, 5)
axarr.plot(x, em, 'r', label='em')
axarr.errorbar(x, em, yerr=em_std, fmt='r')
axarr.plot(x, g, 'g', label='gibbs')
axarr.errorbar(x, g, yerr=g_std, fmt='g')
axarr.plot(x, gf, 'b', label='gibbs fuzzy')
axarr.errorbar(x, gf, yerr=gf_std, fmt='b')
axarr.set_xlabel('P', fontsize=14)
axarr.set_ylabel('Trustworthiness', fontsize=14)
axarr.legend(loc='lower center', fancybox=True, shadow=True, ncol=3)
axarr.set_title('Algorithm trustworthiness, {} objects, {} sources'.format(obj_number, s_number), fontsize=15, fontweight='bold')
axarr.axis([-0.1, 1.1, 0.4, 1.])
plt.show()

In [12]:
f, axarr = plt.subplots()
f.set_size_inches(14, 5)
axarr.plot(x, em_it, 'r', label='em')
axarr.errorbar(x, em_it, yerr=em_it_std, fmt='r')
axarr.plot(x, g, 'g', label='gibbs')
axarr.errorbar(x, g_it, yerr=g_it_std, fmt='g')
axarr.plot(x, gf_it, 'b', label='gibbs fuzzy')
axarr.errorbar(x, gf_it, yerr=gf_it_std, fmt='b')
axarr.set_xlabel('P', fontsize=14)
axarr.set_ylabel('iter number', fontsize=14)
axarr.legend(loc='lower center', fancybox=True, shadow=True, ncol=3)
axarr.set_title('Iter number, {} objects, {} sources, cluster size: 2'.format(obj_number, s_number), fontsize=15, fontweight='bold')
# axarr.axis([-0.1, 1.1, 0.4, 0.9])
plt.show()